In [ ]:
# 400 meter coordinate accuracy
# 1924 earliest year
import pandas as pd
import geopandas as gpd

# from dask import dataframe as dd

# import dask_geopandas

pd.set_option("display.max_columns", None)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', None)

In [ ]:
# read in cleaned parquet
df_path = input("Enter the file path: ")

# df = dd.read_parquet(df_path)
# df = gpd.read_parquet(df_path)
df = pd.read_parquet(df_path)

In [ ]:
# read in eco_map.geojson
map_path = input("Enter the file path: ")

map = gpd.read_file(map_path)

In [ ]:
map.head()

In [ ]:
map.crs

In [ ]:
df = gpd.GeoDataFrame(df)

In [ ]:
df["geometry"] = gpd.points_from_xy(df["decimalLongitude"], df["decimalLatitude"])

In [ ]:
df = df.set_geometry("geometry")

df = df.set_crs("EPSG:4326")

In [ ]:
df.crs

In [ ]:
df = df.drop(["decimalLatitude", "decimalLongitude"], axis=1)

In [ ]:
df = df.sjoin(map, predicate="within")

In [ ]:
# ECOREGIONS ONLY, NO FRESHWATER OR SOIL
# remove rows with <NA> in unique_id from df
df = df[df["unique_id"] != "<NA>"]

In [ ]:
del map, map_path

In [ ]:
df

In [ ]:
# df = df.drop(["index_right", "geometry", "name", "TYPE"], axis=1)

# freshwater
# df = df.drop(["index_right", "geometry", "name", "rights"], axis=1)

# soil
df = df.drop(
    [
        "index_right",
        "geometry",
        "rights",
        "dominant_soil_type_percentage",
        "soil_texture",
        "soil_slope",
        "soil_id",
        "FAOSOIL",
    ],
    axis=1,
)

In [ ]:
df

In [ ]:
# unique_regions = (
#     df.groupby("species")
#     .apply(
#         lambda x: pd.Series(
#             {
#                 "unique_id": list(x["unique_id"].unique()),
#                 "rights": list(
#                     x[["unique_id", "rights"]].drop_duplicates(
#                         "unique_id", keep="first"
#                     )["rights"]
#                 ),
#             }
#         )
#     )
#     .reset_index()
# )

# freshwater
# unique_regions = (
#     df.groupby("species")
#     .apply(
#         lambda x: pd.Series(
#             {
#                 "id": list(x["id"].unique()),
#             }
#         )
#     )
#     .reset_index()
# )

# soil
unique_regions = (
    df.groupby("species")
    .apply(
        lambda x: pd.Series(
            {
                "id": list(x["id"]),
                "specific_soil_name": list(x["specific_soil_name"]),
                "dominant_soil_name": list(x["dominant_soil_name"]),
            }
        )
    )
    .reset_index()
)

In [ ]:
unique_species = df.drop_duplicates(subset=["species"])

# ecoregions
# unique_species = unique_species.drop(["unique_id", "rights"], axis=1)

# freshwater
# unique_species = unique_species.drop(["id"], axis=1)

# soil
unique_species = unique_species.drop(
    ["specific_soil_name", "dominant_soil_name", "id"], axis=1
)

In [ ]:
df = pd.merge(unique_regions, unique_species, on="species", how="inner")

In [ ]:
df

In [ ]:
# df = df.rename(
#     columns={"species": "scientific_name", "unique_id": "observed_ecoregions"}
# )

# freshwater
# df = df.rename(columns={"species": "scientific_name", "id": "freshwater_ecoregions"})

# soil
df = df.rename(columns={"species": "scientific_name", "id": "soil_id"})

In [ ]:
df

In [ ]:
# ECOREGIONS ONLY

# remove None values from rights column
df["rights"] = df["rights"].apply(lambda x: [i for i in x if i is not None])

In [ ]:
df

In [ ]:
df_spatial_path = input("Enter the file path: ")


# write df to parquet file using pandas to_parquet
df.to_parquet(df_spatial_path)